In [1]:
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver import Chrome
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

delay = 3
browser = Chrome("./chromedriver")
browser.implicitly_wait(delay)
browser.get("https://www.youtube.com/channel/UCyDZ65ZwrCD3sDYyrAGV66g/videos") # 박막례할머니 동영상 페이지로 들어감
browser.maximize_window()
body = browser.find_element_by_tag_name('body') # body태그 가져오기

num_of_pagedowns = 22
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1 #video페이지의 스크롤을 충분히 밑으로 내려놓고

In [2]:
html0 = browser.page_source
html = BeautifulSoup(html0, 'html.parser')
video_list2 = html.select('div #dismissable')

youtube_url = 'http://www.youtube.com'

park_url = []

#썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요 를 저장하기 위한 리스트
thumbnail_list = []
category_list = []
upload_date_list = []
video_length_list = []
title_list = []
view_count_list = []
likes_count_list = []
dislikes_count_list = []


print("video_list2의 길이:",len(video_list2))
for i in range(len(video_list2)):
    url = youtube_url + video_list2[i].find('a',{'id': 'thumbnail'})['href']
    park_url.append(url) # 모든 영상의 url을 youtube_url에 붙여서 완전한 url로 만들고 park_url이라는 리스트에 넣어줌

video_list2의 길이: 41


In [3]:
for video in video_list2:
    video_length = video.select_one("ytd-thumbnail-overlay-time-status-renderer.ytd-thumbnail > span.style-scope")
    video_length = video_length.text.replace("\n","").strip()
    video_length_list.append(video_length)
    print(video_length)

4:26
7:25
2:52
3:55
6:26
5:41
5:17
3:04
7:41
6:29
6:14
5:26
6:15
4:01
2:59
5:49
4:43
4:38
3:08
4:34
6:51
3:04
2:44
4:06
1:46
2:11
5:04
2:52
3:12
3:33
3:19
3:36
6:10
3:16
7:03
4:06
3:40
3:13
4:43
2:13
2:38


In [4]:
# 현재로서는 썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요, 만 크롤링 하면 될듯!

# park = pd.DataFrame({'thumbnail': [],
#                      'title' : [],
#                      'category': [],
#                      'view_count': [],
#                      'likes_count': [],
#                      'dislikes_count':[],
#                      'video_length':[],
#                      'upload_date': [],
#                      }) #총 8개의 정보

In [ ]:
for i in range(len(video_list2)):
    #여기까지는 video페이지에서 수집할 수 있는 썸네일, 제목을 가져온다.
    thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져와라 # 1
    title = video_list2[i].find('a',{'id':'video-title'}).text # 2 
    
    # 여기서부터 영상 클릭해서 들어간 페이지
    click_url = park_url[i]
    browser.get(click_url)
    time.sleep(1.5) # 페이지를 넘어가야하므로 로딩을 기다려야함
    # body = browser.find_element_by_tag_name('body')
    browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기를 눌러주자
    time.sleep(3)
    html0 = browser.page_source
    html = BeautifulSoup(html0, 'html.parser')
   
    # 여기서는 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링한다 
    category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
    view_count = html.find('span', {'class': 'view-count'}).text # 4
    
    # 좋아요와 싫어요는 완전히 똑같은 태그에 들어있다.
    # <파이썬 정규 표현식을 지원하는 >re모듈을 사용해보자
    likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 5
    'aria-label':re.compile('좋아요')}).text+'개'
    dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 6
    'aria-label':re.compile('싫어요')}).text+'개'
    upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text # 8
    
    print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)
    
    thumbnail_list.append(thumbnail)
    title_list.append(title)
    category_list.append(category)
    view_count_list.append(view_count)
    likes_count_list.append(likes_count)
    dislikes_count_list.append(dislikes_count)
    upload_date_list.append(upload_date)

0 . 썸네일 https://i.ytimg.com/vi/Zi591bP19jg/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAgJWQjWzN-Ht617lWaHOn7EflJFA 제목: 스트릿출신 아기개냥이 눈만 마주쳐도 애교가 콸콸콸~💦 카테고리 동물 조회수 조회수 40,485회 좋아요수 2.3천개 싫어요수 8개 게시일 2020. 1. 6.
1 . 썸네일 https://i.ytimg.com/vi/u-OG5LWmzTg/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCx9L_K3k_IUWsHJ7jTt-t8KFyB2A 제목: [아깽이 비밀기지] 고양이가 사라졌다면 여기를 찾아보세요! 카테고리 동물 조회수 조회수 18,942회 좋아요수 1천개 싫어요수 12개 게시일 2020. 1. 3.
2 . 썸네일 https://i.ytimg.com/vi/BxPuIX_lS3w/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLB0HpIgRVmUS3qDpVrBx_i9J6nY3A 제목: 냥줍한 개냥이랑 새해 해돋이 보기🌄 카테고리 동물 조회수 조회수 17,687회 좋아요수 1.2천개 싫어요수 3개 게시일 2019. 12. 31.
3 . 썸네일 https://i.ytimg.com/vi/KO6PesQBfHo/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDBcp_P3VhoyzMIG3n3pF6j46wEOA 제목: 세상 귀여운 아기고양이들의 젤리 공개 영상🐾 카테고리 동물 조회수 조회수 10,644회 좋아요수 769개 싫어요수 2개 게시일 2019. 12. 30.
4 . 썸네일 https://i.ytimg.com/vi/2Cs7RGs8NfY/hqdefault.jpg?sqp=-

35 . 썸네일 https://i.ytimg.com/vi/36rT6qtxbM4/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCcqJ1I8XjwzvOzQIJG3-VK1BRh8g 제목: 귀여운 길고양이 시루 냥줍 스토리 카테고리 동물 조회수 조회수 9,440회 좋아요수 668개 싫어요수 4개 게시일 2019. 10. 1.


In [ ]:
import pandas as pd
insert_data = pd.DataFrame({
                    "항목" : ["썸네일","제목","카테고리","조회수","좋아요 수","싫어요 수","업로드날짜","영상길이"]
                    'thumbnail': thumbnail_list,
                    'title' : title_list,
                    'category': category_list,
                    'view_count': view_count_list,
                    'likes_count': likes_count_list,
                    'dislikes_count':dislikes_count_list,
                    'upload_date': upload_date_list,
                    'video_length' : video_length_list
                    }) 
insert_data

In [ ]:
insert_data.to_csv("cat.csv",encoding="utf8")

In [ ]:
# 소련여자, 박막례, 최고다윽박, ondo온도, 오킹, 오늘의 하늘,튜브몬,박병창